# Direct classification from tile to label

In [1]:
import os
import re
import shutil
from skimage import io
from PIL import Image

from torchvision import transforms
from torch.utils.data import DataLoader
from skimage import io
import torch
import glob
import torch
from torch import nn
from torchvision import models
from torch.optim import Adam
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset

import torchvision
from torchvision import datasets, models, transforms

from itertools import cycle

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import auc
import copy

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import glob
# import staintools
from PIL import ImageOps

## Prepare the label

In [110]:
df = pd.read_excel('data_file.xlsx')

In [111]:
# List of all the labels: 
# Comment all except the one you want to analyze 

# Numerical Labels: 
label = "Ki67 (%)"
label = "TILS (%)"
label = "PR (%)"
label = "MYC"
label = "FOXA1"
label = "HIF1A"
label = "ROR.S..Subtype.Only."
label = "ROR.P..Subtype...Proliferation."

# Binary Labels: 
label = "PR status" # Positive / Negative
label = "PAM50 subtype" # Luminal A / Luminal B

In [112]:
df = df[['ID_number', label]]
df = df.dropna(subset=[label])
df

,ID_number,PAM50 subtype
0,SCAN_0054,LumB
1,SCAN_0055,LumB
2,SCAN_0056,LumB
3,SCAN_0057,LumB
4,SCAN_0058,LumB
...,...,...
276,SCAN_0240,LumA
277,SCAN_0241,LumA
278,SCAN_0469,LumA
279,SCAN_0470,LumA


In [113]:
df = df[df[label].notna()]

if label == "PAM50 subtype":
    df = df[df[label].isin(['LumA', 'LumB'])]
    df[label] = df[label].replace({'LumA': 0, 'LumB': 1})
    
elif label == "PR status":
    df[label] = df[label].replace({'Negative': 0, 'Positive': 1})

# else label is numerical
else:
    df[label] = pd.to_numeric(df[label], errors='coerce')

In [114]:
np.unique(df[label], return_counts=True)

(array([0, 1]), array([141, 135]))

In [115]:
q20 = df[label].quantile(0.20)
q80 = df[label].quantile(0.80)

# Create new dataframe by filtering values below 20th percentile and above 80th percentile
df_extreme = df[(df[label] <= q20) | (df[label] >= q80)].copy()

# Set labels: 0 for values below or equal to 20th percentile, 1 for values above or equal to 80th percentile
df_extreme.loc[df_extreme[label] <= q20, label] = -100
df_extreme.loc[df_extreme[label] >= q80, label] = 100

df_extreme.replace({-100: 0, 100: 1}, inplace=True)

In [116]:
# Create out of distribution set for numerical values:
if label != "PR status" and label != "PAM50 subtype":
    q20 = df[label].quantile(0.20)
    q80 = df[label].quantile(0.80)
    q30 = df[label].quantile(0.30)
    q70 = df[label].quantile(0.70)

    # Create new dataframe by filtering values between 20th and 30th percentile and between 70th and 80th percentile
    df_ood = df[((df[label] <= q30) & (df[label] > q20)) | ((df[label] < q80) & (df[label] >= q70))].copy()
    df_ood.loc[df_ood[label] <= q30, label] = -100
    df_ood.loc[df_ood[label] >= q70, label] = 100
    df_ood.replace({-100: 0, 100: 1}, inplace=True)

# Create the different datasets

## Create the original dataset with all the images

In [119]:
class BiopsyDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, df, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.df = df
        self.image_filenames = []
        self.labels = []

        for image_filename in glob.glob(os.path.join(self.root_dir, '*.jpeg')):
            ID_number = image_filename.split('/')[-1].split('.tif')[0]
            if ID_number in df.ID_number.tolist():
                self.image_filenames.append(image_filename)
                self.labels.append(df[df['ID_number'] == ID_number][label].values[0])

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image = io.imread(self.image_filenames[idx])
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label
    
    def get_label_indices(self):
        # Get indices for labels 0 and 1
        zeros_indices = [i for i, x in enumerate(self.labels) if x == 0]
        ones_indices = [i for i, x in enumerate(self.labels) if x == 1]
        return zeros_indices, ones_indices
    

# Define transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(128),
    transforms.ToTensor(),
])

# Create train, validation and test sets
train_df, temp_df = train_test_split(df_extreme, test_size=0.3, random_state=41, stratify=df_extreme[label])
valid_df, test_df = train_test_split(temp_df, test_size=0.33, random_state=41, stratify=temp_df[label])

# Create datasets
train_dataset = BiopsyDataset("/storage/Chloe/zoom_20_512", train_df, transform=transform)
valid_dataset = BiopsyDataset("/storage/Chloe/zoom_20_512", valid_df, transform=transform)
test_dataset = BiopsyDataset("/storage/Chloe/zoom_20_512", test_df, transform=transform)
if label != "PR status" and label != "PAM50 subtype":
    test_dataset_ood = BiopsyDataset("/storage/Chloe/zoom_20_512", df_ood, transform=transform)

In [120]:
print(len(train_dataset))
print(len(valid_dataset))
print(len(test_dataset))
if label != "PR status" and label != "PAM50 subtype":
    print(len(test_dataset_ood))

158810
48608
29295


## Create balanced datasets with labels

In [121]:
class SubsetDataset(torch.utils.data.Dataset):
    def __init__(self, subset_indices, original_dataset):
        self.subset_indices = subset_indices
        self.original_dataset = original_dataset

    def __getitem__(self, index):
        original_index = self.subset_indices[index]
        return self.original_dataset[original_index]

    def __len__(self):
        return len(self.subset_indices)

In [122]:
def balanced_data(dataset):
    zero_index, one_index = dataset.get_label_indices()
    min_class_size = min(len(zero_index), len(one_index))
    np.random.seed(42)  # Set the random seed for reproducibility
    balanced_indices_class0 = np.random.choice(zero_index, size=min_class_size, replace=False)
    balanced_indices_class1 = np.random.choice(one_index, size=min_class_size, replace=False)
    balanced_indices = np.concatenate([balanced_indices_class0, balanced_indices_class1])
    balanced_dataset = SubsetDataset(balanced_indices, dataset)
    return balanced_dataset

In [123]:
balanced_train_dataset = balanced_data(train_dataset)
balanced_valid_dataset = balanced_data(valid_dataset)
balanced_test_dataset = balanced_data(test_dataset)
if label != "PR status" and label != "PAM50 subtype":
    balanced_test_ood_dataset = balanced_data(test_dataset_ood)

In [124]:
print(len(balanced_train_dataset))
print(len(balanced_valid_dataset))
print(len(balanced_test_dataset))
if label != "PR status" and label != "PAM50 subtype":
    print(len(balanced_test_ood_dataset))

89058
18788
20722


## Create transformed dataset with label

In [22]:
class TransformedDataset(Dataset):
    def __init__(self, base_dataset, transform):
        self.base_dataset = base_dataset
        self.transform = transform

    def __getitem__(self, idx):
        image, label = self.base_dataset[idx]
        return self.transform(image), label

    def __len__(self):
        return len(self.base_dataset)

In [23]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
    transforms.Resize(128),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [24]:
transformed_train_dataset = TransformedDataset(balanced_train_dataset, transform)
transformed_valid_dataset = TransformedDataset(balanced_valid_dataset, transform)
transformed_test_dataset = TransformedDataset(balanced_test_dataset, transform)
if label != "PR status" and label != "PAM50 subtype":
    transformed_test_ood_dataset = TransformedDataset(balanced_test_ood_dataset, transform)

In [25]:
# Create dataloaders
train_dataloader = DataLoader(transformed_train_dataset, batch_size=8, shuffle=True)
valid_dataloader = DataLoader(transformed_valid_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(transformed_test_dataset, batch_size=8, shuffle=True)
if label != "PR status" and label != "PAM50 subtype":
    test_ood_dataloader = DataLoader(transformed_test_ood_dataset, batch_size=8, shuffle=True)

# Classification 

In [ ]:
def evaluate_model(model, dataloader):
    model.eval()  
    true_labels = []
    predictions = []
    prediction_probs = []
    
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        with torch.no_grad():
            outputs = model(inputs).view(-1)
        preds = (torch.sigmoid(outputs) > 0.5).int()
        true_labels.extend(labels.int().tolist())
        predictions.extend(preds.tolist())
        prediction_probs.extend(torch.sigmoid(outputs).tolist())
    
    acc = accuracy_score(true_labels, predictions)
    auc = roc_auc_score(true_labels, prediction_probs)
    f1 = f1_score(true_labels, predictions)
    conf_matrix = confusion_matrix(true_labels, predictions)
    plt.figure(figsize=(7,5))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()
    print(f"AUC-ROC Score: {auc}")
    print(f"F1 Score: {f1}")
    return acc


In [26]:
def train_model(model, train_dataloader, valid_dataloader, criterion, optimizer, num_epochs, file_path):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_epoch = 0
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in train_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs).view(-1)
            loss = criterion(outputs, labels.float())
            preds = (torch.sigmoid(outputs) > 0.5).int()
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            running_corrects += (preds == labels).sum().item()

        epoch_loss = running_loss / len(train_dataloader.dataset)
        epoch_acc = running_corrects / len(train_dataloader.dataset)
        valid_acc = evaluate_model(model, valid_dataloader)

        if valid_acc > best_acc:
            best_epoch = epoch
            best_acc = valid_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), file_path)

        print(f'Epoch {epoch}, Loss: {epoch_loss:.4f}, Acc: {epoch_acc}, Valid Acc: {valid_acc}')

    print(f'Best epoch at {best_epoch}')
    model.load_state_dict(best_model_wts)
    return model

## Resnet 50

In [ ]:
class MyModelResnet(nn.Module):
    def __init__(self, base_model):
        super(MyModelResnet, self).__init__()
        self.base_model = base_model

    def forward(self, x):
        x = self.base_model(x)
        return x

model_base = models.resnet50(pretrained=True)

# Modify the final layer of the base models
num_ftrs = model_base.fc.in_features
model_base.fc = nn.Linear(num_ftrs, 1)
model = MyModelResnet(model_base)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer1 = Adam(model.parameters(), lr=0.001)

In [ ]:
best_classifier = train_model(
    model, 
    train_dataloader, 
    valid_dataloader, 
    criterion, 
    optimizer1, 
    num_epochs=20, 
    file_path = '/storage/Chloe/final_models_classification/direct_classification_Ki67_2_Resnet_split41.pth'
)

In [ ]:
test_acc = evaluate_model(best_classifier, test_dataloader)
print(f'Test accuracy: {test_acc}')

In [ ]:
if label != "PR status" and label != "PAM50 subtype":
    test_ood_acc = evaluate_model(best_classifier, test_ood_dataloader)
    print(f'Test ood accuracy: {test_ood_acc}')

## Densenet121

In [ ]:
class MyModelDensenet(nn.Module):
    def __init__(self, base_model):
        super(MyModelDensenet, self).__init__()
        self.upsample = nn.Upsample((32, 32))
        self.base_model = base_model

    def forward(self, x):
        x = self.upsample(x)
        x = self.base_model(x)
        return x


model_base = models.densenet121(pretrained=True)
num_ftrs = model_base.classifier.in_features
model_base.classifier = nn.Linear(num_ftrs, 1)
model = MyModelDensenet(model_base)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer1 = Adam(model.parameters(), lr=0.001)

In [ ]:
best_classifier = train_model(
    model, 
    train_dataloader, 
    valid_dataloader, 
    criterion, 
    optimizer1, 
    num_epochs=20, 
    file_path='/storage/Chloe/final_models_classification/direct_classification_Ki67_2_densenet_split41.pth'
)


In [ ]:
test_acc = evaluate_model(best_classifier, test_dataloader)
print(f'Test accuracy: {test_acc}')

In [ ]:
if label != "PR status" and label != "PAM50 subtype":
    test_ood_acc = evaluate_model(best_classifier, test_ood_dataloader)
    print(f'Test ood accuracy: {test_ood_acc}')

## Inception V3

In [ ]:
class MyModelInceptionV3(nn.Module):
    def __init__(self, base_model):
        super(MyModelInceptionV3, self).__init__()
        self.upsample = nn.Upsample((299, 299))
        self.base_model = base_model

    def forward(self, x):
        x = self.upsample(x)
        if self.training and self.base_model.aux_logits:
            x, _ = self.base_model(x)
        else:
            x = self.base_model(x)
        return x



model_base = models.inception_v3(pretrained=True, aux_logits=True)
num_ftrs = model_base.fc.in_features
model_base.fc = nn.Linear(num_ftrs, 1)
num_aux_ftrs = model_base.AuxLogits.fc.in_features
model_base.AuxLogits.fc = nn.Linear(num_aux_ftrs, 1)
model = MyModelInceptionV3(model_base)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer1 = Adam(model.parameters(), lr=0.001)

In [ ]:
best_classifier = train_model(
    model, 
    train_dataloader, 
    valid_dataloader, 
    criterion, 
    optimizer1, 
    num_epochs=20, 
    file_path='/storage/Chloe/final_models_classification/direct_classification_Ki67_2_Inception_split41.pth'
)


In [ ]:
test_acc = evaluate_model(best_classifier, test_dataloader)
print(f'Test accuracy: {test_acc}')

In [ ]:
if label != "PR status" and label != "PAM50 subtype":
    test_ood_acc = evaluate_model(best_classifier, test_ood_dataloader)
    print(f'Test ood accuracy: {test_ood_acc}')